<a href="https://colab.research.google.com/github/maryclareok/3classesfireandsmoke/blob/main/sosesworkmay2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install --upgrade tensorflow


This script is performing several steps of a machine learning workflow, including data loading, preprocessing, model training, and prediction. Here's a breakdown:

1. **Data Loading**: The script starts by defining the path to the directory where the data files are stored (`dir_path`), and the subdirectories within that directory (`sub_dirs`). It then initializes an empty list (`dfs`) to store the dataframes. The script loops over each subdirectory, and for each .csv file in the subdirectory, it reads the file into a pandas DataFrame and appends it to the list. After all files have been read, it concatenates the dataframes into a single dataframe (`df`).

2. **Data Preprocessing**: The script defines the features to be used for clustering (`features`). It then converts non-numeric values in these features to NaN, and drops the rows with NaN values. After that, it scales the features using `StandardScaler` from sklearn, which standardizes the features to have zero mean and unit variance.

3. **Model Definition and Training**: The script defines an autoencoder model using Keras. An autoencoder is a type of neural network used for learning efficient codings of input data. It consists of an encoder, which learns a compressed representation of the input data, and a decoder, which learns to reconstruct the original data from the compressed representation. The script defines the size of the compressed representation (`encoding_dim`), the input layer (`input_data`), the encoded layer (`encoded`), and the decoded layer (`decoded`). It then defines the autoencoder model (`autoencoder`), compiles it with the Adadelta optimizer and binary crossentropy loss, and trains it on the scaled features.

4. **Dimensionality Reduction**: The script uses the encoder part of the autoencoder to reduce the dimensionality of the data (`X_encoded`).

5. **Clustering**: The script creates a KMeans object (`kmeans`) with a specified number of clusters, fits it to the encoded data, and assigns each data point to a cluster (`df['cluster']`).

So, in summary, this script is loading data from multiple .csv files, preprocessing the data, training an autoencoder model, using the model to reduce the dimensionality of the data, and applying KMeans clustering to the reduced data.

In [17]:
import pandas as pd
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model

# Define the path to the directory
dir_path = '/content/drive/MyDrive/5G-production-dataset'  # replace with your directory path

# Define the subdirectories
sub_dirs = ['Download', 'Netflix', 'Amazon_Prime']

# Define the features for clustering
features = ['Longitude', 'Latitude', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'DL_bitrate', 'UL_bitrate']

# Initialize a list to store dataframes
dfs = []

# Loop through each subdirectory
for sub_dir in sub_dirs:
    # Get the list of .csv files in each subdirectory
    csv_files = [f for f in os.listdir(os.path.join(dir_path, sub_dir)) if f.endswith('.csv')]

    # Loop through each .csv file
    for file in csv_files:
        # Read the .csv file into a pandas DataFrame
        df = pd.read_csv(os.path.join(dir_path, sub_dir, file))

        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes
df = pd.concat(dfs, ignore_index=True)

# Convert non-numeric values to NaN
for feature in features:
    df[feature] = pd.to_numeric(df[feature], errors='coerce')

# Drop the rows with NaN values. Alternatively, you can fill NaN values with a valid numeric value.
df = df.dropna()

# Preprocess the data (e.g., normalization)
scaler = StandardScaler()
X = scaler.fit_transform(df[features])

# Define the size of the encoded representations
encoding_dim = 32  # this is the size of our encoded representations

# Define the input layer
input_data = Input(shape=(len(features),))

# Define the encoded layer
encoded = Dense(encoding_dim, activation='relu')(input_data)

# Define the decoded layer
decoded = Dense(len(features), activation='sigmoid')(encoded)

# Define the autoencoder model
autoencoder = Model(input_data, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# Train the autoencoder model
history = autoencoder.fit(X, X, epochs=50, batch_size=256, shuffle=True)

# Use the encoder part of the autoencoder to reduce the dimensionality of the data
encoder = Model(input_data, encoded)
X_encoded = encoder.predict(X)

# Create a KMeans object
kmeans = KMeans(n_clusters=3)  # You can change the number of clusters

# Fit the model to the encoded data
kmeans.fit(X_encoded)

# Get the cluster assignments for each data point
df['cluster'] = kmeans.labels_


Epoch 1/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.7555
Epoch 2/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.7472
Epoch 3/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.7378
Epoch 4/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.7261
Epoch 5/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.7128
Epoch 6/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.7008
Epoch 7/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.6881
Epoch 8/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.6732
Epoch 9/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6593
Epoch 10/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6448
Epoch 11/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6290
Epoch 12/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6143
Epoch 13/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5992
Epoch 14/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5832
Epoch 15/50
519/519 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


#silhouette score

The Silhouette Score is a measure of how well each datapoint \(x_i\) lies within its assigned cluster. It ranges from -1 to 1, where a higher value indicates that the datapoint is well matched to its own cluster and poorly matched to neighboring clusters.

In the context of your project, a Silhouette Score of 0.22227174 suggests that the datapoints (in this case, the base stations) are, on average, not very densely packed in their assigned clusters. This could mean that there might be some overlap between the clusters, or that the datapoints within each cluster are spread out.

Here's a general guideline for interpreting the Silhouette Score:

- Values close to 1 suggest that the datapoints are well clustered.
- Values close to 0 suggest that the datapoints are on or very close to the decision boundary between two neighboring clusters.
- Negative values suggest that those datapoints might have been assigned to the wrong cluster.

So, in your case, a Silhouette Score of 0.22227174 suggests that there might be room for improving the clustering, perhaps by using a different number of clusters, using a different clustering algorithm, or by preprocessing the data differently.

In [18]:
# from sklearn.metrics import silhouette_score

# # Compute the silhouette score
# silhouette = silhouette_score(X_encoded, df['cluster'])

# print('Silhouette Score:', silhouette)


#SAVE MODEL
This code is saving the trained models to disk so that they can be loaded and used later without needing to be retrained. Here’s a breakdown of what each line is doing:

autoencoder.save('/content/drive/MyDrive/autoencoder_model.h5'): This line is saving the trained autoencoder model to a file named ‘autoencoder_model.h5’ in the directory ‘/content/drive/MyDrive’. The ‘.h5’ extension indicates that the model is being saved in the HDF5 format, which is a common format for storing large amounts of numerical data like the weights of a neural network.
encoder.save('/content/drive/MyDrive/encoder_model.h5'): This line is saving the trained encoder model (which is part of the autoencoder) to a file named ‘encoder_model.h5’ in the same directory. Again, the model is being saved in the HDF5 format.
import pickle: This line is importing the ‘pickle’ module, which is a Python module used for serializing (i.e., converting to a byte stream) and deserializing Python objects.
with open('/content/drive/MyDrive/kmeans_model.pkl', 'wb') as f:: This line is opening a file named ‘kmeans_model.pkl’ in the directory ‘/content/drive/MyDrive’ in binary write mode (‘wb’). The ‘.pkl’ extension indicates that the file is a pickle file, which is a common format for serializing Python objects.
pickle.dump(kmeans, f): This line is using the ‘dump’ function from the ‘pickle’ module to serialize the trained KMeans model and write it to the file.
So, in summary, this code is saving the trained autoencoder, encoder, and KMeans models to disk so that they can be loaded and used later without needing to be retrained.


In [19]:
# Save the trained models
autoencoder.save('/content/drive/MyDrive/autoencoder_model.h5')
encoder.save('/content/drive/MyDrive/encoder_model.h5')

# Save the KMeans model
import pickle
with open('/content/drive/MyDrive/kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

# Group the data by cluster and calculate the mean of the features
This code is calculating the mean (average) values of the features for each cluster and printing them out. These mean values can be considered as the “centers” of the clusters in the feature space.

Here’s a breakdown of what each line is doing:

cluster_centers = df.groupby('cluster')[features].mean(): This line is grouping the DataFrame df by the ‘cluster’ column, selecting the columns specified in features, and calculating the mean of these columns for each cluster. The result is a new DataFrame where the index is the cluster number and the columns are the mean values of the features for each cluster.
print(cluster_centers): This line is printing the cluster_centers DataFrame. Each row in this DataFrame corresponds to a cluster, and the values in the row are the mean values of the features for that cluster.
So, in summary, this code is providing a summary of each cluster by calculating and printing the mean values of the features for each cluster. This can help you understand the characteristics of each cluster.

In [20]:
# Group the data by cluster and calculate the mean of the features
cluster_centers = df.groupby('cluster')[features].mean()

# Print the cluster centers
print(cluster_centers)


         Longitude   Latitude       RSRP       RSRQ       SNR        CQI  \
cluster                                                                    
0        -8.479674  51.900141 -88.581101 -13.540216 -1.235857   8.252456   
1        -8.437100  51.913664 -81.827136 -10.161336  8.458620  12.610125   
2        -8.398574  51.890772 -99.968956 -13.607344  2.028992  10.716020   

              RSSI    DL_bitrate  UL_bitrate  
cluster                                       
0       -73.656635   1405.645304   20.350910  
1       -71.152584  11518.900758   58.761302  
2       -91.971379  26689.776187   72.991202  


# Get the base stations in each cluster
This code is iterating over each cluster created by the KMeans algorithm and printing out the base stations (represented by ‘CellID’) that belong to each cluster.

Here’s a breakdown of what each line is doing:

for i in range(kmeans.n_clusters): This line is starting a loop that will iterate over each cluster. The number of clusters is determined by the n_clusters attribute of the kmeans object.
print(f"Cluster {i}:") This line is printing the cluster number for each iteration. The {i} inside the string will be replaced by the current cluster number.
print(df[df['cluster'] == i]['CellID']) This line is printing the ‘CellID’ of the base stations that belong to the current cluster. It does this by subsetting the DataFrame df to only include rows where the ‘cluster’ column is equal to the current cluster number i, and then selecting the ‘CellID’ column.
So, in summary, this code is printing out the base stations that belong to each cluster, which can help you understand how the base stations are grouped based on the KMeans clustering algorithm.

In [21]:
# Get the base stations in each cluster
for i in range(kmeans.n_clusters):
    print(f"Cluster {i}:")
    print(df[df['cluster'] == i]['CellID'])


Cluster 0:
359       13
575        2
584       12
622        2
623        2
          ..
153431     2
153436     2
153437     2
153438     2
153439     2
Name: CellID, Length: 35123, dtype: int64
Cluster 1:
0         12
1         12
2         12
3         12
4         12
          ..
165864    11
165919    11
166030    11
166141    11
166211    11
Name: CellID, Length: 51873, dtype: int64
Cluster 2:
6         12
7         12
8         12
10        12
26        12
          ..
166652    11
166653    11
166654    11
166655    11
166656    11
Name: CellID, Length: 45806, dtype: int64


#GMM
The choice between using KMeans or Gaussian Mixture Models (GMM) depends on the specific requirements of your project and the nature of your data.

**KMeans** is a simple and fast algorithm that works well when the clusters are spherical and roughly the same size. It assigns each data point to exactly one cluster, which makes it easy to interpret the results. However, KMeans assumes that clusters are spherical and have similar variances, which might not always be the case.

On the other hand, **GMM** is a more flexible method that allows for mixed membership of points to clusters. In GMM, a point belongs to each cluster to a different degree, based on the probability of the point being generated from each cluster's Gaussian distribution. This makes GMM more capable of handling clusters of different shapes and sizes. However, GMM is more computationally intensive than KMeans and might be harder to interpret due to the mixed membership of points to clusters.

In your case, we started with KMeans because it's a simple and commonly used algorithm that serves as a good starting point. After applying KMeans, we used the Silhouette Score to evaluate the quality of the clusters. The Silhouette Score suggested that there might be some overlap between the clusters, which is why we considered using GMM to find the overlapping regions.

Using GMM at this point allows us to take advantage of its ability to handle overlapping clusters, which might provide more insights into your data. However, whether to use KMeans, GMM, or any other clustering algorithm should be decided based on your specific requirements and the nature of your data.


In [22]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3)
gmm.fit(X_encoded)

# Get the cluster assignments for each data point
df['cluster'] = gmm.predict(X_encoded)

# Get the probabilities of each data point belonging to each cluster
probs = gmm.predict_proba(X_encoded)


#PROBABILITY OF EACH BASE STATION BEING OVERLAPPED BY ANOTHER CLUSTER
The output you're seeing is a DataFrame that represents the probabilities of each data point (or row in your original dataset) belonging to each cluster.

- The columns `Cluster 0`, `Cluster 1`, and `Cluster 2` represent the three clusters that your data points have been grouped into by the Gaussian Mixture Model (GMM).
- The values in these columns are the probabilities that a given data point belongs to that cluster. For example, a value of `1.000000e+00` in `Cluster 0` for a particular row means that the GMM model is almost certain (with a probability of 1) that this data point belongs to `Cluster 0`.
- The `CellID` column is just the `CellID` from your original dataset. It's included in this DataFrame for reference, so you can see which cell each row of probabilities corresponds to.

The code you've posted calculates these probabilities using the `predict_proba` method of the GMM, which returns the probability of each sample belonging to each cluster. It then filters the DataFrame to only include rows where more than one cluster has a probability greater than a certain threshold (0.5 in your case), indicating that these base stations could potentially belong to more than one cluster.

Please note that the probabilities are in scientific notation. For example, `1.000000e+00` is equivalent to 1, `3.566801e-67` is a very small number close to 0, and `5.992325e-168` is even closer to 0. .

In [23]:
# Get the probabilities of each data point belonging to each cluster
probs = gmm.predict_proba(X_encoded)

# Convert the probabilities to a DataFrame
probs_df = pd.DataFrame(probs, columns=[f'Cluster {i}' for i in range(kmeans.n_clusters)])

# Add the 'CellID' column to the probabilities DataFrame
probs_df['CellID'] = df['CellID']

# Set a threshold for the minimum probability to consider a base station as part of a cluster
threshold = 0.5  # You can adjust this value based on your requirements

# Find the base stations that have a probability greater than the threshold for more than one cluster
overlapping_base_stations = probs_df[probs_df.gt(threshold).sum(axis=1) > 1]

print(overlapping_base_stations)


           Cluster 0     Cluster 1      Cluster 2  CellID
0       1.000000e+00  3.566801e-67   0.000000e+00    12.0
1       1.000000e+00  1.529159e-62  5.992325e-168    12.0
2       1.000000e+00  1.669245e-62  4.703392e-168    12.0
3       1.000000e+00  1.667903e-62  4.696484e-168    12.0
4       1.000000e+00  1.374847e-51  3.378590e-157    12.0
...              ...           ...            ...     ...
132797  4.226672e-28  0.000000e+00   1.000000e+00    11.0
132798  6.375775e-28  0.000000e+00   1.000000e+00    11.0
132799  2.584201e-28  0.000000e+00   1.000000e+00    11.0
132800  1.000000e+00  0.000000e+00   0.000000e+00    11.0
132801  1.000000e+00  0.000000e+00   0.000000e+00    11.0

[114503 rows x 4 columns]


In summary: for the base station 12 it belongs to cluster 0 because it has a probabilty of 1 , it definetely doesnt belong to cluster 1 because it has 0 probabilty but its getting interference from cluster 2 and the same for base station with cell id 11


#Plotting the Training Loss of the Autoencoder

In [24]:
import matplotlib.pyplot as plt

# Assuming 'history' is the output of the fit function
# history = autoencoder.fit(X, X, epochs=50, batch_size=256, shuffle=True)

plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()


#Silhouette Score for Different Numbers of Clusters

In [25]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
cluster_range = range(2, 10)  # Adjust based on your requirements

for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(X_encoded)
    silhouette_scores.append(silhouette_score(X_encoded, kmeans.labels_))

plt.plot(cluster_range, silhouette_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.show()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KeyboardInterrupt: 

#Bar Plot of Cluster Centers

In [26]:
# Assuming 'cluster_centers' is a DataFrame containing the mean values of the features for each cluster
# cluster_centers = df.groupby('cluster')[features].mean()

cluster_centers.plot(kind='bar', figsize=(10, 6))
plt.title('Cluster Centers')
plt.show()


#This script will calculate the Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Index, and Inertia for your clusters. It will also generate a scatter plot, dendrogram, heatmap of the distance matrix, and parallel coordinates plot for your data.



In [1]:
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram, linkage
import seaborn as sns
from pandas.plotting import parallel_coordinates

# Calculate metrics
davies_bouldin = davies_bouldin_score(X_encoded, df['cluster'])
calinski_harabasz = calinski_harabasz_score(X_encoded, df['cluster'])
inertia = kmeans.inertia_

# Print metrics
print(f'Davies-Bouldin Index: {davies_bouldin}')
print(f'Calinski-Harabasz Index: {calinski_harabasz}')
print(f'Inertia: {inertia}')

# Scatter plot
plt.figure(figsize=(10, 7))
plt.scatter(X_encoded[:, 0], X_encoded[:, 1], c=df['cluster'])
plt.xlabel('Encoded feature 1')  # Replace with your actual feature name
plt.ylabel('Encoded feature 2')  # Replace with your actual feature name
plt.title('Scatter Plot of Clusters')
plt.show()

# Dendrogram
linked = linkage(X_encoded, 'single')
plt.figure(figsize=(10, 7))
dendrogram(linked)
plt.title('Dendrogram of Clusters')
plt.show()

# Heatmap of distance matrix
sns.heatmap(X_encoded)
plt.title('Heatmap of Distance Matrix')
plt.show()

# Parallel coordinates plot
plt.figure(figsize=(10, 7))
parallel_coordinates(df[features + ['cluster']], 'cluster')
plt.title('Parallel Coordinates Plot of Clusters')
plt.show()

# Bar plot of metrics
metrics = {'Davies-Bouldin Index': davies_bouldin, 'Calinski-Harabasz Index': calinski_harabasz, 'Inertia': inertia}
plt.bar(metrics.keys(), metrics.values())
plt.title('Clustering Metrics')
plt.show()


NameError: name 'X_encoded' is not defined